In [1]:
import numpy as np
import seaborn as sns
from copy import deepcopy
import os
from pathlib import Path
from tqdm.auto import tqdm
from tqdm.contrib.concurrent import process_map

/opt/anaconda3/envs/kaggle_tpu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
root = Path("/home/edu/code/google_fast_or_slow/data/npz_pad")
collection = "layout/xla"
ctype = "random"

In [3]:
def vec_to_int(vec: np.ndarray) -> np.ndarray:
    # Powers of 7: [1, 7, 49, 343, 2401, 16807]
    powers_of_7 = np.array([7**i for i in range(6)])
    return np.dot(vec, powers_of_7).astype(np.int32)


def int_to_vec(integers: np.ndarray) -> np.ndarray:
    # Create an empty array of shape (N, 6) to store the results
    vectors = np.empty((len(integers), 6), dtype=np.int64)

    # Divide by powers of 7 and take the remainder to find each digit
    for i in range(6):
        vectors[:, i] = integers % 7
        integers //= 7

    return vectors.astype(np.int32)


def compress_configs(node_configs):
    vecs = node_configs.reshape(-1, 6).astype(np.int32) + 1
    ints = vec_to_int(vecs)
    ints = ints.reshape(node_configs.shape[0], node_configs.shape[1], 3)
    return ints


def decompress_configs(node_configs):
    ints = node_configs.astype(np.int32).reshape(-1)
    vecs = int_to_vec(ints)
    vecs = vecs.reshape(node_configs.shape[0], -1, 18) - 1
    return vecs
    
def prune_graph(data):
    print("Pruning graph...")
    new_data = deepcopy(dict(data))
    print("Original graph has {} nodes and {} edges".format(data["node_feat"].shape[0], data["edge_index"].shape[0]))
    in_edge_index = data["edge_index"][np.isin(data["edge_index"], data["node_config_ids"]).any(1)]

    in_node_ids = np.unique(in_edge_index)
    assert len(set(data["node_config_ids"]) - set(in_node_ids)) == 0
    lookup = np.ones(data["node_feat"].shape[0]) * -1
    lookup[in_node_ids] = np.arange(in_node_ids.shape[0])

    in_node_feats = data["node_feat"][in_node_ids, :]
    in_node_opcode = data["node_opcode"][in_node_ids]
    in_edge_index = lookup[in_edge_index]
    in_node_config_ids = lookup[data["node_config_ids"]]

    new_data["node_feat"] = in_node_feats
    new_data["node_opcode"] = in_node_opcode
    new_data["edge_index"] = in_edge_index
    new_data["node_config_ids"] = in_node_config_ids
    print("New graph has {} nodes and {} edges".format(new_data["node_feat"].shape[0], new_data["edge_index"].shape[0]))
    return new_data


In [4]:
def remove_dupplicated_node_configs(data):
    reshaped_config_feat = data["node_config_feat"].reshape(data["node_config_feat"].shape[0], -1) + 2 # avoid zeros
    positional_array = np.random.random(reshaped_config_feat.shape[1])  # multiply each value by its position to avoid removing permutations by accident
    reshaped_values = (reshaped_config_feat * positional_array[None, :]).sum(1)
    is_equal_matrix = reshaped_values[None, :] == reshaped_values[:, None] # quadratic matrix of all pairwise equalities
    # is_equal_matrix[np.triu_indices(is_equal_matrix.shape[0], 0)] = 0 # only get diagonal to avoid remove twice
    is_equal_matrix = np.tril(is_equal_matrix, -1) # only get diagonal to avoid remove twice
    to_remove_ids = np.unique(np.where(is_equal_matrix)[0])
    print("Removing {} duplicated node configs out of {}".format(to_remove_ids.shape[0], data["node_config_feat"].shape[0]))
    data["config_runtime"] = np.delete(data["config_runtime"], to_remove_ids)
    data["node_config_feat"] = np.delete(data["node_config_feat"], to_remove_ids, axis=0)
    return data

In [5]:
dst_dir = root / f"{collection}_pruned" / ctype

for split in ["train", "valid", "test"]:
    print("Loading {} data...".format(split))
    split_src_dir = root / collection / ctype / split
    split_dst_dir = dst_dir / split
    split_dst_dir.mkdir(parents=True, exist_ok=True)

    def _process_one_npz(npz_path):
        data = dict(np.load(str(npz_path), allow_pickle=True))
        data = prune_graph(data)
        if split == "train":
            data = remove_dupplicated_node_configs(data)
        data["node_config_feat"] = compress_configs(data["node_config_feat"])
        np.savez_compressed(split_dst_dir / npz_path.name, **data)
    
    process_map(_process_one_npz, list(split_src_dir.glob("*.npz")), max_workers=3)

    # for npz_path in tqdm(list(split_src_dir.glob("*.npz"))):
        # print(npz_path)
        # data = dict(np.load(str(npz_path), allow_pickle=True))
        # data = prune_graph(data)
        # if split == "train":
        #     data = remove_dupplicated_node_configs(data)
        # data["node_config_feat"] = compress_configs(data["node_config_feat"])
        # np.savez_compressed(split_dst_dir / npz_path.name, **data)
        # if split == "valid":
        #     data = remove_dupplicated_node_configs(data)
        #     dedup_dst_dir = Path(str(split_dst_dir).replace("valid", "valid_dedup"))
        #     dedup_dst_dir.mkdir(parents=True, exist_ok=True)
        #     np.savez(dedup_dst_dir / npz_path.name, **data)

Loading train data...


  0%|          | 0/69 [00:00<?, ?it/s]

Pruning graph...
Original graph has 13502 nodes and 22209 edges
New graph has 2143 nodes and 2137 edges
Pruning graph...
Original graph has 490 nodes and 729 edges
New graph has 65 nodes and 56 edges
Removing 39 duplicated node configs out of 1632
Pruning graph...
Original graph has 4185 nodes and 7209 edges
New graph has 224 nodes and 200 edges


  1%|▏         | 1/69 [00:00<00:32,  2.07it/s]

Pruning graph...
Original graph has 6328 nodes and 12014 edges
New graph has 426 nodes and 375 edges
Removing 459 duplicated node configs out of 100040
Pruning graph...
Original graph has 13342 nodes and 21709 edges
New graph has 1820 nodes and 1622 edges
Removing 39 duplicated node configs out of 1152
Pruning graph...
Original graph has 5809 nodes and 10345 edges
New graph has 594 nodes and 599 edges
Removing 39 duplicated node configs out of 7208
Pruning graph...
Original graph has 15642 nodes and 25387 edges
New graph has 3878 nodes and 3394 edges
Removing 19 duplicated node configs out of 20
Pruning graph...
Original graph has 8847 nodes and 14797 edges
New graph has 811 nodes and 806 edges
Removing 39 duplicated node configs out of 7440
Pruning graph...
Original graph has 17472 nodes and 27130 edges
New graph has 4211 nodes and 4097 edges
Removing 39 duplicated node configs out of 156
Pruning graph...
Original graph has 40332 nodes and 71912 edges
New graph has 7304 nodes and 6804

...


  0%|          | 0/8 [00:00<?, ?it/s]

Pruning graph...
Pruning graph...
Original graph has 5279 nodes and 8694 edges
Original graph has 5810 nodes and 10345 edges
New graph has 582 nodes and 589 edges
New graph has 594 nodes and 599 edges
Pruning graph...
Original graph has 490 nodes and 749 edges
New graph has 118 nodes and 116 edges
Pruning graph...
Original graph has 43615 nodes and 73881 edges
New graph has 3142 nodes and 3089 edges
Pruning graph...
Pruning graph...
Original graph has 23363 nodes and 38984 edges
New graph has 6140 nodes and 6482 edges
Original graph has 24790 nodes and 32709 edges
New graph has 6411 nodes and 4774 edges
Pruning graph...
Original graph has 41522 nodes and 72902 edges
New graph has 6959 nodes and 6466 edges


 25%|██▌       | 2/8 [00:00<00:01,  3.83it/s]

Pruning graph...
Original graph has 43615 nodes and 73881 edges
New graph has 3142 nodes and 3089 edges


100%|██████████| 8/8 [00:00<00:00, 11.33it/s]


In [ ]:
import pandas as pd
root = Path("/home/edu/code/google_fast_or_slow/data/npz_all/npz")
collection = "layout/xla_pruned"
ctype = "default"

split_src_dir = root / collection / ctype / "train"
df = []
for path in tqdm(list(split_src_dir.glob("*.npz"))):
    data = dict(np.load(str(path), allow_pickle=True))
    times = data['config_runtime']
    if len(times) < 2:
        continue
    times = np.sort(times)
    deltas = times[1:] - times[:-1]
    stats = {"file": path.stem, "median": np.median(deltas), "mean": np.mean(deltas), "std": np.std(deltas), "min": np.min(deltas), "max": np.max(deltas), "p10": np.percentile(deltas, 10), "p90": np.percentile(deltas, 90)}
    df.append(stats)
    # ids = np.where(data["node_opcode"][data["node_config_ids"].astype(int)] == 75)
    # if ids:
    #     ids = ids[0]
    #     data["node_config_feat"] = decompress_configs(data["node_config_feat"])
    #     counter.extend(data["node_config_feat"][:, ids][:, :, [2, 3, 4, 5, 8, 9, 10, 11, 14, 15, 16, 17]].reshape(-1))
df_default = pd.DataFrame(df)

In [ ]:
df_random.iloc[:, 1:].mean()

In [ ]:
df_default.iloc[:, 1:].mean()

In [ ]:
stats # random

In [ ]:
stats # default

In [ ]:
np.unique(counter, return_counts=True)

In [ ]:
data = dict(np.load("/home/edu/code/google_fast_or_slow/data/npz_pad/layout/xla/random/train/resnet_v2_50_batch_16.npz"))

In [ ]:
data["node_config_feat"] = decompress_configs(data["node_config_feat"])

In [ ]:
seqs = []
for i in data["node_config_feat"][:, 0, :]:
    seqs.append(str(i))

In [ ]:
np.unique(seqs, return_counts=True)

In [ ]:
data["node_opcode"][497]

In [ ]:
list(enumerate(data["node_feat"][497, :]))

In [ ]:
list(enumerate(data["node_feat"][497, :]))

In [ ]:
data["node_config_ids"][0]

In [ ]:
data["node_opcode"][data["node_config_ids"][0:450].astype(int)]

In [ ]:
np.unique(data["node_opcode"][data["node_config_ids"].astype(int)], return_counts=True)

In [ ]:
data["node_config_feat"].shape

In [ ]:
def inspect_node(idx):
    print("Node idx: {}".format(idx))
    print("Node opcode: {}".format(data["node_opcode"][idx]))
    print("Node config id: {}".format(data["node_config_ids"][idx]))
    print("Node config feat: {}".format(data["node_config_feat"][idx]))
    print("Node feat: {}".format(data["node_feat"][idx]))

In [8]:
import torch
x = torch.load("/home/edu/code/google_fast_or_slow/outputs_xla_default/checkpoint-12000/pytorch_model.bin")

In [9]:
x.keys()

odict_keys(['embedding_op.weight', 'embedding_layout_cfg.weight', 'linear.linear.weight', 'linear.linear.bias', 'linear.cross_attn.temperature', 'linear.attn.linear1.weight', 'linear.attn.linear1.bias', 'linear.attn.linear2.weight', 'linear.attn.linear2.bias', 'convs.0.conv.lin_l.weight', 'convs.0.conv.lin_l.bias', 'convs.0.conv.lin_r.weight', 'convs.0.attn.linear1.weight', 'convs.0.attn.linear1.bias', 'convs.0.attn.linear2.weight', 'convs.0.attn.linear2.bias', 'convs.0.cross_attn.temperature', 'convs.1.conv.lin_l.weight', 'convs.1.conv.lin_l.bias', 'convs.1.conv.lin_r.weight', 'convs.1.attn.linear1.weight', 'convs.1.attn.linear1.bias', 'convs.1.attn.linear2.weight', 'convs.1.attn.linear2.bias', 'convs.1.cross_attn.temperature', 'classifier.weight', 'classifier.bias'])

In [14]:
[(_, x[_]) for _ in x.keys() if "cross_attn.temperature" in _]

[('linear.cross_attn.temperature', tensor(0.6401, device='cuda:0')),
 ('convs.0.cross_attn.temperature', tensor(0.3068, device='cuda:0')),
 ('convs.1.cross_attn.temperature', tensor(0.3477, device='cuda:0'))]

In [1]:
import numpy as np
np.random.randint(1, 5)

3

In [2]:
data = np.load("/home/edu/code/google_fast_or_slow/data/npz_pad/layout/xla_pruned_compressed/default/test/3e7156ac468dfb75cf5c9615e1e5887d.npz")

In [4]:
data["config_runtime"]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [1]:
import pickle
with open("/home/edu/code/google_fast_or_slow/outputs_xla_random/fold_0/scaler.pkl", "rb") as f:
    data = pickle.load(f)

In [5]:
import numpy as np
import pandas as pd
from pathlib import Path
import shutil

In [15]:
paths = [
    "/home/edu/code/google_fast_or_slow/outputs_probs/layout:xla_pruned:random:val_probs_cross_128_701.csv",
    "/home/edu/code/google_fast_or_slow/outputs_probs/layout:xla_pruned:random:val_probs_no_cross_685.csv",
]
weights = [0.5, 0.5]
root = Path("/home/edu/code/google_fast_or_slow/data/npz_pad/layout/")
collection = "xla_pruned_compressed"
search = "random"
split = "valid"

In [16]:
all_probs = []
for p, w in zip(paths, weights):
    df = pd.read_csv(p)
    filenames = df["ID"].tolist()
    gts = df["gts"].apply(lambda x: list(map(float, x.split(";")))).tolist()
    probs = df["probs"].apply(lambda x: list(map(float, x.split(";")))).tolist()
    probs = [np.array(_) * w for _ in probs]
    all_probs.append(probs)

avg_probs = []
for i in range(len(filenames)):
    avg_probs.append(np.stack([model_prob[i] for model_prob in all_probs]).sum(0))

In [17]:
from scipy.stats import kendalltau
score = [kendalltau(p, gt).statistic for p, gt in zip(avg_probs, gts)]
score

[0.9787235221144981,
 0.6396126601435936,
 0.49751865369094184,
 0.9267971867094805,
 0.7788089689258385,
 0.7192630813017497,
 0.35548091897764134]

In [18]:
np.mean(score)

0.6994578559805349

In [19]:
# create pseudo labels
src_dir = root / collection / search / split
dst_dir = root / collection / search / f"{split}_pseudo"
dst_dir.mkdir(exist_ok=True)
for file, pseudo_runtime in zip(filenames, avg_probs):
    file = file.split(":")[-1] + ".npz"
    data = dict(np.load(str(src_dir / file), allow_pickle=True))
    assert len(data["config_runtime"]) == len(pseudo_runtime)
    data["config_runtime"] = pseudo_runtime
    data = remove_dupplicated_node_configs(data)
    np.savez_compressed(str(dst_dir / file), **data)


Removing 39 duplicated node configs out of 19040
Removing 39 duplicated node configs out of 4688
Removing 39 duplicated node configs out of 5288
Removing 39 duplicated node configs out of 5704
Removing 39 duplicated node configs out of 7248
Removing 39 duplicated node configs out of 17360
Removing 39 duplicated node configs out of 481


In [7]:

prediction_indices = []
for pred_prob in avg_probs:
    prediction = np.argsort(pred_prob)
    prediction_indices.append(";".join([str(int(e)) for e in prediction]))

In [13]:
import os

submission_df = pd.DataFrame.from_dict(
    {
        "ID": filenames,
        "TopConfigs": prediction_indices,
    }
)

submission_df.to_csv(
    os.path.join(
        "/home/edu/code/google_fast_or_slow/outputs_csv",
        f"layout:xla:{search}:submission.csv",
    ),
    index=False,
)